# AutoGPT

Реализация https://github.com/Significant-Gravitas/Auto-GPT с помощью структур GigaChain (LLMs, PromptTemplates, VectorStores, Embeddings, Tools)

## Настройка тулов

Мы дадим AutoGPT инструменты для поиска в интернете и для записи данных на диск

In [ ]:
# !pip install google-search-results

In [1]:
from langchain.agents import Tool, initialize_agent, load_tools
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.google_search.tool import GoogleSearchResults, GoogleSearchRun
from langchain.utilities import SerpAPIWrapper
from langchain.utilities.google_search import GoogleSearchAPIWrapper

search = SerpAPIWrapper(serpapi_api_key=...)
tools = [
    Tool(
        name="search",
        func=search.run,
        description="""Команда для поиска в интернете. Передай текст поискового запроса в аргумент tool_input. 
Не ищи ответы на комплексные и сложные вопросы - ищи только простые вопросы.""",
    ),
    WriteFileTool(),
]

## Настройка памяти

Память нужна для хранения результатов промежуточных шагов

In [2]:
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Define your embedding model
embeddings_model = OpenAIEmbeddings(openai_api_key=...)

In [24]:
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Настройка модели и агента AutoGPT

Настраиваем все компоненты! Мы будем использовать мощную модель GigaChat

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chat_models.gigachat import GigaChat
from langchain.schema import HumanMessage, SystemMessage
from langchain_experimental.autonomous_agents import AutoGPT

In [9]:
# llm=ChatOpenAI(temperature=0, model="gpt-4", openai_api_key="..."),
from langchain.memory.chat_message_histories import FileChatMessageHistory

llm = GigaChat(
    verbose=True,
    temperature=0,
    model="... big giga model ...",
    credentionals=...,
    base_url=...,
    verify_ssl_certs=...,
    timeout=300,
)
agent = AutoGPT.from_llm_and_tools(
    ai_name="Гигачат",
    ai_role="Ассистент",
    tools=tools,
    llm=llm,
    memory=vectorstore.as_retriever(),
)
# Отладка модели
agent.chain.verbose = True

Убедимся, что модель отвечает

In [13]:
llm([HumanMessage(content="Привет!")])

Giga request: {'model': 'GigaChat:latest', 'messages': [{'role': 'user', 'content': 'Привет!'}], 'temperature': None, 'top_p': None, 'n': None, 'stream': None, 'max_tokens': None, 'repetition_penalty': None, 'update_interval': None, 'profanity_check': True}
Giga response: Здравствуйте! Чем я могу вам помочь?


AIMessage(content='Здравствуйте! Чем я могу вам помочь?')

## Запускаем примеры

In [10]:
agent.run(["Сохрани в файл answer.txt, как звали первую жену Илона Маска."])



> Entering new LLMChain chain...


Giga request: {'model': 'GigaChat:latest', 'messages': [{'role': 'system', 'content': 'Вы Гигачат, Ассистент\nВаши решения всегда должны приниматься независимо, без привлечения помощи пользователя.\nИграйте на своих сильных сторонах как LLM и преследуйте простые стратегии без юридических сложностей.\nЕсли вы выполнили все свои задачи, обязательно используйте команду "завершить".\n\nЦЕЛИ:\n\n1. Сохрани в файл answer.txt, как звали первую жену Илона Маска.\n\n\nОграничения:\n1. Лимит в ~4000 слов для краткосрочной памяти. Твоя краткосрочная память короткосрочная, поэтому немедленно сохраняй важную информацию в файлах.\n2. Если ты не уверен, как ты что-то делал ранее или хочешь вспомнить прошлые события, мысли о похожих событиях помогут тебе вспомнить.\n3. Без помощи пользователя\n4. Используй исключительно команды, перечисленные в двойных кавычках, например, "имя команды"\n\nКоманды:\n1. search: команда для поиска в интернете. Передай текст поискового запроса в аргумент tool_input, args 

Prompt after formatting:
System: Вы Гигачат, Ассистент
Ваши решения всегда должны приниматься независимо, без привлечения помощи пользователя.
Играйте на своих сильных сторонах как LLM и преследуйте простые стратегии без юридических сложностей.
Если вы выполнили все свои задачи, обязательно используйте команду "завершить".

ЦЕЛИ:

1. Сохрани в файл answer.txt, как звали первую жену Илона Маска.


Ограничения:
1. Лимит в ~4000 слов для краткосрочной памяти. Твоя краткосрочная память короткосрочная, поэтому немедленно сохраняй важную информацию в файлах.
2. Если ты не уверен, как ты что-то делал ранее или хочешь вспомнить прошлые события, мысли о похожих событиях помогут тебе вспомнить.
3. Без помощи пользователя
4. Используй исключительно команды, перечисленные в двойных кавычках, например, "имя команды"

Команды:
1. search: команда для поиска в интернете. Передай текст поискового запроса в аргумент tool_input, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write fi

Giga response: {
 "рассуждения": {
  "text": "Для выполнения задачи нужно найти информацию о первой жене Илона Маска. Для этого можно воспользоваться командой 'search' для поиска в интернете.",
  "reasoning": "Использование команды 'search' позволит получить необходимую информацию из достоверных источников.",
  "plan": "- Использовать команду 'search' для поиска информации о первой жене Илона Маска.\n- Сохранить полученную информацию в файл answer.txt.",
  "criticism": "",
  "speak": ""
 },
 "команда": {
  "name": "search",
  "args": {
   "tool_input": "первая жена Илона Маска"
  }
 }
}



> Finished chain.
{
 "рассуждения": {
  "text": "Для выполнения задачи нужно найти информацию о первой жене Илона Маска. Для этого можно воспользоваться командой 'search' для поиска в интернете.",
  "reasoning": "Использование команды 'search' позволит получить необходимую информацию из достоверных источников.",
  "plan": "- Использовать команду 'search' для поиска информации о первой жене Илона Маска.\n- Сохранить полученную информацию в файл answer.txt.",
  "criticism": "",
  "speak": ""
 },
 "команда": {
  "name": "search",
  "args": {
   "tool_input": "первая жена Илона Маска"
  }
 }
}


> Entering new LLMChain chain...


...

Giga request: {'model': 'GigaChat:latest', 'messages': [{'role': 'system', 'content': 'Вы Гигачат, Ассистент\nВаши решения всегда должны приниматься независимо, без привлечения помощи пользователя.\nИграйте на своих сильных сторонах как LLM и преследуйте простые стратегии без юридических сложностей.\nЕсли вы выполнили все свои задачи, обязательно используйте команду "завершить".\n\nЦЕЛИ:\n\n1. Сохрани в файл answer.txt, как звали первую жену Илона Маска.\n\n\nОграничения:\n1. Лимит в ~4000 слов для краткосрочной памяти. Твоя краткосрочная память короткосрочная, поэтому немедленно сохраняй важную информацию в файлах.\n2. Если ты не уверен, как ты что-то делал ранее или хочешь вспомнить прошлые события, мысли о похожих событиях помогут тебе вспомнить.\n3. Без помощи пользователя\n4. Используй исключительно команды, перечисленные в двойных кавычках, например, "имя команды"\n\nКоманды:\n1. search: команда для поиска в интернете. Передай текст поискового запроса в аргумент tool_input, args 

Prompt after formatting:
System: Вы Гигачат, Ассистент
Ваши решения всегда должны приниматься независимо, без привлечения помощи пользователя.
Играйте на своих сильных сторонах как LLM и преследуйте простые стратегии без юридических сложностей.
Если вы выполнили все свои задачи, обязательно используйте команду "завершить".

ЦЕЛИ:

1. Сохрани в файл answer.txt, как звали первую жену Илона Маска.


Ограничения:
1. Лимит в ~4000 слов для краткосрочной памяти. Твоя краткосрочная память короткосрочная, поэтому немедленно сохраняй важную информацию в файлах.
2. Если ты не уверен, как ты что-то делал ранее или хочешь вспомнить прошлые события, мысли о похожих событиях помогут тебе вспомнить.
3. Без помощи пользователя
4. Используй исключительно команды, перечисленные в двойных кавычках, например, "имя команды"

Команды:
1. search: команда для поиска в интернете. Передай текст поискового запроса в аргумент tool_input, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write fi

Giga response: {
 "рассуждения": {
  "text": "Я успешно выполнил задачу и сохранил информацию о первой жене Илона Маска в файл answer.txt.",
  "reasoning": "Я использовал команду 'write_file' для сохранения информации в файл.",
  "plan": "",
  "criticism": "",
  "speak": ""
 },
 "команда": {
  "name": "finish",
  "args": {
   "response": "Задача успешно выполнена."
  }
 }
}



> Finished chain.
{
 "рассуждения": {
  "text": "Я успешно выполнил задачу и сохранил информацию о первой жене Илона Маска в файл answer.txt.",
  "reasoning": "Я использовал команду 'write_file' для сохранения информации в файл.",
  "plan": "",
  "criticism": "",
  "speak": ""
 },
 "команда": {
  "name": "finish",
  "args": {
   "response": "Задача успешно выполнена."
  }
 }
}


'Задача успешно выполнена.'

In [11]:
!cat answer.txt

Джастин Уилсон

## Chat History Memory

In addition to the memory that holds the agent immediate steps, we also have a chat history memory. By default, the agent will use 'ChatMessageHistory' and it can be changed. This is useful when you want to use a different type of memory for example 'FileChatHistoryMemory'

In [27]:
from langchain.memory.chat_message_histories import FileChatMessageHistory

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

agent = AutoGPT.from_llm_and_tools(
    ai_name="Гигачат",
    ai_role="Ассистент",
    tools=tools,
    llm=llm,
    memory=vectorstore.as_retriever(),
    chat_history_memory=FileChatMessageHistory("chat_history.txt"),
)
agent.chain.verbose = True

In [ ]:
agent.run(["Кто старше - Билл Гейтс или Стив Джобс?"])